In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read in data
df = pd.read_csv('allegations_202007271729.csv') 

In [3]:
# Filter data to only necessary colummns
df = df[['complaint_id', 'year_received', 'precinct', 'fado_type']]
# Fix nulls and datatypes
df = df[df['precinct'].notnull()]
df['precinct'] = df['precinct'].astype(int)
df['precinct'] = df['precinct'].astype('string')
df['fado_type'] = df['fado_type'].astype('string')
# Remove 2020 data
df = df[df['year_received'] <= 2019]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33330 entries, 0 to 33357
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   complaint_id   33330 non-null  int64 
 1   year_received  33330 non-null  int64 
 2   precinct       33330 non-null  string
 3   fado_type      33330 non-null  string
dtypes: int64(2), string(2)
memory usage: 1.3 MB


In [4]:
df['precinct'].unique()

<StringArray>
[  '78',   '67',   '79',   '77',   '81',   '73',   '90',   '75',  '120',
   '10',   '25',  '110',   '83',   '46',    '9',   '28',    '1',   '68',
   '70',   '72',  '122',  '113',  '101',  '100',    '7',   '13',   '71',
   '23',  '112',   '69',   '32',   '94',   '63',   '76',   '52',   '47',
   '62',   '42',   '84',  '115',  '109',    '5',   '88',   '14',  '103',
  '107',  '104',   '40',  '108',   '50',   '45',   '18',  '102',  '114',
   '19',   '20',  '106',  '105',   '17',   '60',   '34',   '44',   '48',
   '41',   '43',  '111',   '49',   '26',   '30',   '33',    '6',   '61',
  '123',  '121',   '66',   '24',    '0',   '22', '1000']
Length: 79, dtype: string

In [5]:
df[df['precinct'].isin(['0', '1000'])]['precinct'].value_counts()

precinct
0       21
1000     3
Name: count, dtype: Int64

In [6]:
# Count
before = len(df)

# Removing precinct 0 and 1000, not real nypd precincts
df = df[~df['precinct'].isin(['0', '1000'])]

print(f"Removed {before - len(df)} rows.")

Removed 24 rows.


In [7]:
# Group data by year, precinct, & fado_type -> report complaint counts
counts = (
    df.groupby(['year_received', 'precinct', 'fado_type']).size().reset_index(name='complaint_count')
)
counts

,year_received,precinct,fado_type,complaint_count
0,1985,63,Force,1
1,1985,67,Abuse of Authority,1
2,1985,71,Abuse of Authority,1
3,1985,71,Force,1
4,1985,83,Abuse of Authority,1
...,...,...,...,...
4791,2019,9,Offensive Language,3
4792,2019,90,Abuse of Authority,69
4793,2019,90,Discourtesy,2
4794,2019,90,Force,1


In [8]:
# Pivot data so that each fado_type has its own column with # of complaints
wide = counts.pivot_table(
    index=['year_received', 'precinct'],
    columns='fado_type',
    values='complaint_count',
    fill_value=0
).reset_index()

# Add column for total # of complaints (sum of fado_type columns)
wide['total_complaints'] = wide[['Abuse of Authority', 'Discourtesy', 'Force', 'Offensive Language']].sum(axis=1)

wide.columns.name = None

wide

,year_received,precinct,Abuse of Authority,Discourtesy,Force,Offensive Language,total_complaints
0,1985,63,0.0,0.0,1.0,0.0,1.0
1,1985,67,1.0,0.0,0.0,0.0,1.0
2,1985,71,1.0,0.0,1.0,0.0,2.0
3,1985,83,1.0,1.0,1.0,0.0,3.0
4,1986,14,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
1764,2019,84,1.0,0.0,0.0,0.0,1.0
1765,2019,88,4.0,1.0,1.0,0.0,6.0
1766,2019,9,7.0,2.0,5.0,3.0,17.0
1767,2019,90,69.0,2.0,1.0,0.0,72.0


In [9]:
# Add a zero-count for missing (year, precinct) pairs

# Pull list of all years and precincts
all_years = np.arange(df['year_received'].min(), df['year_received'].max() + 1)
all_precincts = df['precinct'].unique()

# Create all possible combinations of years and precincts
all_combos = pd.MultiIndex.from_product([all_years, all_precincts],names=['year_received', 'precinct'])

# Reindex to include missing combos (fill with 0)
wide = wide.set_index(['year_received', 'precinct']).reindex(all_combos, fill_value=0).reset_index()

# Update total_complaints
wide['total_complaints'] = wide[['Abuse of Authority', 'Discourtesy', 'Force', 'Offensive Language']].sum(axis=1)

wide

,year_received,precinct,Abuse of Authority,Discourtesy,Force,Offensive Language,total_complaints
0,1985,78,0.0,0.0,0.0,0.0,0.0
1,1985,67,1.0,0.0,0.0,0.0,1.0
2,1985,79,0.0,0.0,0.0,0.0,0.0
3,1985,77,0.0,0.0,0.0,0.0,0.0
4,1985,81,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2690,2019,123,6.0,0.0,0.0,0.0,6.0
2691,2019,121,15.0,3.0,8.0,0.0,26.0
2692,2019,66,0.0,0.0,0.0,0.0,0.0
2693,2019,24,5.0,0.0,1.0,0.0,6.0


In [10]:
# Export pandas dataframe to .json
wide.to_json('complaints_by_precinct.json', orient='records')